In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torchsummary
from torchvision import transforms
import time
from sklearn.metrics import classification_report

In [4]:
match_folder=np.loadtxt('/home/sumins/workspace/washout classification/base_data/condition_match_folder.txt',dtype=str)
match_folder=match_folder.tolist()

In [5]:
len(match_folder)

165

In [6]:
import ast
range_path='/home/sumins/workspace/washout classification/base_data/liverrange.txt'
range_value_path='/home/sumins/workspace/washout classification/base_data/liverrange_value.txt'

liverrange=[]
liverrange_value=[]
with open (range_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange.append(ast.literal_eval(line))

with open (range_value_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange_value.append(ast.literal_eval(line))

In [7]:
same=0
diff=0
diff_index=[]
for index, i in enumerate(liverrange_value):
    if i[0]==i[1]==i[2]==i[3]:
        same+=1
    else:
        diff+=1
        diff_index.append(index)
        

In [8]:
print("same ",same)
print("diff ", diff)

same  146
diff  19


In [9]:
type(liverrange_value[2][3])

int

In [31]:
def findnear(mylist,num):
    near_value_plus=0
    near_value_minus=0
    near_value=0
    for i in mylist:
        if i <=num:
            near_value_minus=i
        if i>=num:
            near_value_plus=i
            break
    if (num-near_value_minus)<=(near_value_plus-num):
        near_value=near_value_minus
    else:
        near_value=near_value_plus
    return near_value
        
r_data=[]
a_data=[]
p_data=[]
d_data=[]
for i,k in enumerate(liverrange_value):
    r=[]
    a=[]
    p=[]
    d=[]
    r_new=[]
    a_new=[]
    p_new=[]
    d_new=[]
    liver=[]
    if k[0]==k[1]==k[2]==k[3]:
        for num in range(k[0]):
            r_new.append(liverrange[i][0]+num)
            a_new.append(liverrange[i][2]+num)
            p_new.append(liverrange[i][4]+num)
            d_new.append(liverrange[i][6]+num)
    else:
        min_phase=liverrange_value[i].index(min(liverrange_value[i]))
        myrange=liverrange[i][:]
        for j in range(liverrange_value[i][0]):
            r.append((j)/(liverrange_value[i][0]))
        for j in range(liverrange_value[i][1]):
            a.append((j)/(liverrange_value[i][1]))
        for j in range(liverrange_value[i][2]):
            p.append((j)/(liverrange_value[i][2]))
        for j in range(liverrange_value[i][3]):
            d.append((j)/(liverrange_value[i][3])) 
        liver.append(r)
        liver.append(a)
        liver.append(p)
        liver.append(d)  
        
        for q in liver[min_phase]:
            r_new.append(findnear(r,q))
            a_new.append(findnear(a,q))
            p_new.append(findnear(p,q))
            d_new.append(findnear(d,q))
        

        for w in range(len(r_new)):
            r_new[w]=int(r_new[w]*(liverrange_value[i][0])+myrange[0])
            a_new[w]=int(a_new[w]*(liverrange_value[i][1])+myrange[2])
            p_new[w]=int(p_new[w]*(liverrange_value[i][2])+myrange[4])
            d_new[w]=int(d_new[w]*(liverrange_value[i][3])+myrange[6])
 
            
    r_data.append(r_new)
    a_data.append(a_new)
    p_data.append(p_new)
    d_data.append(d_new)

In [32]:
savepath1='/home/sumins/workspace/washout classification/min_slice/pre.txt'
savepath2='/home/sumins/workspace/washout classification/min_slice/arterial.txt'
savepath3='/home/sumins/workspace/washout classification/min_slice/portal.txt'
savepath4='/home/sumins/workspace/washout classification/min_slice/delay.txt'
with open (savepath1,'w+') as f:
    for i in r_data:
        f.write(str(i)+'\n')
with open (savepath2,'w+') as f:
    for i in a_data:
        f.write(str(i)+'\n')
with open (savepath3,'w+') as f:
    for i in p_data:
        f.write(str(i)+'\n')
with open (savepath4,'w+') as f:
    for i in d_data:
        f.write(str(i)+'\n')